### Ramdom seed

In [1]:
import torch
import torchvision
from PIL import Image

import os
import random
import matplotlib.pyplot as plt

import torch.nn as nn
from torchvision import datasets,transforms
from torch.utils.data import Dataset, DataLoader

In [2]:
class Custom_Data_class(Dataset):
    def __init__(self, root_path):
        self.root_path = root_path
        self.transforms = transforms.Compose([
                            transforms.Resize((224,224)),
                            transforms.RandomHorizontalFlip(),
                            transforms.ToTensor(),
                             transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
                        ])
        self.labels = [label for label in os.listdir(self.root_path)]
        self.label_folder_path = [os.path.join(self.root_path,label) for label in self.labels]
        self.images = [os.path.join(folder,img_name)  for folder in self.label_folder_path for img_name in os.listdir(folder)]
        random.shuffle(self.images)
        
        
    def __getitem__(self,index):
        image = self.images[index]
        label = image.split("/")[-2]

        img = Image.open(image).convert("RGB")
        # plt.imshow(img)
        # plt.show()
        
        t_img = self.transforms(img)
        b_img = torchvision.transforms.ToPILImage()(t_img)
        # print(b_img)
        # plt.imshow(b_img)
        # plt.show()
        
        label = [i for i,lab in enumerate(self.labels) if lab==label]
        return t_img, label
        
        
    def __len__(self):
        return len(self.images)        

In [3]:
train_Dataset = Custom_Data_class("/home/roufa/Desktop/interview/pytorch/image_classification/hymenoptera_data/train")
val_Dataset = Custom_Data_class("/home/roufa/Desktop/interview/pytorch/image_classification/hymenoptera_data/val")

In [4]:
train_dataloader = DataLoader(train_Dataset, batch_size=4, shuffle=True)
val_dataloader = DataLoader(val_Dataset, batch_size=4, shuffle=True)

In [6]:
class Custom_Model(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(3, 6, (5,5))
        self.conv2 = nn.Conv2d(6, 16,(5,5))
        self.maxpool = nn.MaxPool2d((5,5))
        self.conv3 = nn.Conv2d(16, 32,(5,5))
        self.maxpool1 = nn.MaxPool2d((3,3))
        self.conv4 = nn.Conv2d(32, 64, (5,5))
        self.relu = nn.ReLU()
        
        self.fc1 = nn.Linear(64*9*9, 1024)
        self.fc3 = nn.Linear(1024, 256)
        self.fc4 = nn.Linear(256,2)
            
    def forward(self,x):
        x = self.conv1(x)
        x = self.conv2(x)
        x = self.maxpool(x)
        x = self.relu(x)
        x = self.conv3(x)
        x = self.maxpool1(x)
        x = self.relu(x)
        x = self.conv4(x)
#         print(x.shape)
        
        x = torch.flatten(x,1)
        x = self.fc1(x)
        x = self.fc3(x)
        x = self.fc4(x)
        
        
        return x

In [7]:
model = Custom_Model()
model

Custom_Model(
  (conv1): Conv2d(3, 6, kernel_size=(5, 5), stride=(1, 1))
  (conv2): Conv2d(6, 16, kernel_size=(5, 5), stride=(1, 1))
  (maxpool): MaxPool2d(kernel_size=(5, 5), stride=(5, 5), padding=0, dilation=1, ceil_mode=False)
  (conv3): Conv2d(16, 32, kernel_size=(5, 5), stride=(1, 1))
  (maxpool1): MaxPool2d(kernel_size=(3, 3), stride=(3, 3), padding=0, dilation=1, ceil_mode=False)
  (conv4): Conv2d(32, 64, kernel_size=(5, 5), stride=(1, 1))
  (relu): ReLU()
  (fc1): Linear(in_features=5184, out_features=1024, bias=True)
  (fc3): Linear(in_features=1024, out_features=256, bias=True)
  (fc4): Linear(in_features=256, out_features=2, bias=True)
)

In [8]:
samp = train_Dataset[2][0].unsqueeze(0)
out = model(samp)

In [51]:
#from tqdm.auto import tqdm

In [14]:
epochs = 20
lr = 0.0005

# optimizer = torch.optim.Adam(model.parameters(), lr)
optimizer = torch.optim.SGD(model.parameters(), lr, momentum= 0.8)
criterion = nn.CrossEntropyLoss()
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
losses = []

model = model.to(device)


for epoch in range(epochs):
    
    for i,data in enumerate(train_dataloader):
        inputs,label = data[0].to(device),data[1][0].to(device)
        optimizer.zero_grad()
        output = model(inputs)
        loss = criterion(output,label)
        loss.backward()
        optimizer.step()
        losses.append(loss.item())
#         print(loss.item())

In [15]:
classes = ['bees', 'ants']
correct_pred = {classname: 0 for classname in classes}
total_pred = {classname: 0 for classname in classes}

In [16]:
test_losses = []


for i,data in enumerate(val_dataloader):
    inputs,labels = data[0].to(device),data[1][0].to(device)

    output = model(inputs)
    _, predictions = torch.max(output, 1)


    loss = criterion(output,labels)

    for label, prediction in zip(labels, predictions):
        if label == prediction:
            correct_pred[classes[label]] += 1
        total_pred[classes[label]] += 1

    test_losses.append(loss.item())
        
        
for classname, correct_count in correct_pred.items():
    accuracy = 100 * float(correct_count) / total_pred[classname]
    print(f'Accuracy for class: {classname:5s} is {accuracy:.1f} %')            

Accuracy for class: bees  is 41.0 %
Accuracy for class: ants  is 71.4 %
